In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
class TreeNode:
    def __init__(self, data,output):
        self.data=data
        self.output=output
        self.children={}
        self.index = -1
    
    def child(self,feature_value,obj):
        self.children[feature_value] = obj

In [7]:
class DecisionTreeClassifier:
    def __init__(self):
        self._root=None
    
    def __count_unique(self,Y):
        d={}
        for i in Y:
            if i not in d:
                d[i]=1
            else:
                d[i]+=1
        return d  
    
    def __entropy(self,Y):
            freq_map = self.__count_unique(Y)
            entropy_=0
            total = len(Y)
            for i in freq_map:
                p = freq_map[i]/total
                entropy_ += (-p)*math.log2(p)
            return entropy_
        
    def __gain_ratio(self,X,Y,selected_feature):
        # returns the gain ratio
        info_orig = self.__entropy(Y) # info_orig represents entropy before splitting
        info_f = 0  # info_f represents entropy after splitting upon the selected feature
        split_info = 0
        values = set(X[:,selected_feature])
        #print(values)
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        initial_size = df.shape[0] 
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            info_f += (current_size/initial_size)*self.__entropy(df1[df1.shape[1]-1])
            split_info += (-current_size/initial_size)*math.log2(current_size/initial_size)

        # to handle the case when split info = 0 which leads to division by 0 error
        if split_info == 0 :
            return math.inf

        info_gain = info_orig - info_f
        gain_ratio = info_gain / split_info
        return gain_ratio
        
    def __decision_tree(self,X,Y,features,level,metric,classes): 
        #for pure node
        #base condition1
        if len(set(Y)) == 1:
            print("Level",level)
            output = None
            for i in classes:
                if i in Y:
                    output = i
                    print("Count of",i,"=",len(Y))
                else :
                    print("Count of",i,"=",0)
            if metric == "gain_ratio":
                print("Current Entropy is =  0.0")
            

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)
        
        #for run out of feature
        #base condition2
        if len(features) == 0:
            print("Level",level)
            freq_map = self.__count_unique(Y)
            output = None
            max_count = -math.inf
            for i in classes:
                if i not in freq_map:
                    print("Count of",i,"=",0)
                else :
                    if freq_map[i] > max_count :
                        output = i
                        max_count = freq_map[i]
                    print("Count of",i,"=",freq_map[i])

            if metric == "gain_ratio":
                print("Current Entropy  is =",self.__entropy(Y))
             

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)
        
        #for rest calculate max gain
        #find best feature upon which we can split
        max_gain=-math.inf
        current_gain=0
        for f in features:
            current_gain = self.__gain_ratio(X,Y,f)
            if(current_gain>max_gain):
                max_gain = current_gain
                final_feature = f
        
        #at this time we have final_feature as our splitting attribute and its respective gain is max_gain
        
        print("level",level)
        freq_map = self.__count_unique(Y)
        output = None
        max_count = -math.inf
        for i in classes:
                if freq_map[i] > max_count :
                    output = i
                    max_count = freq_map[i]
                print("Count of",i,"=",freq_map[i])
                
        print("Current Entropy is =",self.__entropy(Y))  
        print("Splitting on feature  X",final_feature+1," with gain ratio ",max_gain,sep="")
        unique_values = set(X[:,final_feature]) 
        # unique_values represents the unique values of the feature selected
        
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        current_node = TreeNode(final_feature,output)
        #finally saving current_node in TreeNode where has to take place
        
        index  = features.index(final_feature)
        features.remove(final_feature)
        #removing the final_feature from feature
        
        #parse on all unique labels in final_feature 
        for i in unique_values:
            # Creating a new dataframe with value of selected feature = i
            df1 = df[df[final_feature] == i]
            # Segregating the X and Y values and recursively calling on the splits
            node = self.__decision_tree(df1.iloc[:,0:df1.shape[1]-1].values,df1.iloc[:,df1.shape[1]-1].values,features,level+1,metric,classes)
            current_node.child(i,node)

        # Adding the removed feature     
        features.insert(index,final_feature)

        return current_node
    
    def __predict_for(self,data,node):
        # predicts the class for a given testing point and returns the answer
        
        # We have reached a leaf node
        if len(node.children) == 0 :
            return node.output
        
        #print("node data",node.data)
        val = data[node.data] # represents the value of feature on which the split was made  
        #print("val",val)
        #print("node children",node.children)
        if val not in node.children :
            return node.output
        
        # Recursively call on the splits
        return self.__predict_for(data,node.children[val])  
    
    def predict(self,X):
        # This function returns Y predicted
        # X should be a 2-D np array
        Y = np.array([0 for i in range(len(X))])
        
        for i in range(len(X)):
            Y[i] = self.__predict_for(X[i],self.__root)
        return Y             
        
    def fit(self,X,Y,metric="gain_ratio"):
        features=[i for i in range(X.shape[1])]
        classes=set(Y)
        level = 0
        self.__root = self.__decision_tree(X,Y,features,level,metric,classes)
        
    def score(self,X,Y):
        # returns the mean accuracy
        Y_pred = self.predict(X)
        count = 0
        for i in range(len(Y_pred)):
            if Y_pred[i] == Y[i]:
                count+=1
                return count/len(Y_pred) 

In [8]:
clf = DecisionTreeClassifier()

x = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

y = np.array([0,
              1,
              1,
              1]) 
clf.fit(x,y)


level 0
Count of 0 = 1
Count of 1 = 3
Current Entropy is = 0.8112781244591328
Splitting on feature  X1 with gain ratio 0.31127812445913283
level 1
Count of 0 = 1
Count of 1 = 1
Current Entropy is = 1.0
Splitting on feature  X2 with gain ratio 1.0
Level 2
Count of 0 = 1
Count of 1 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 1
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 2
Current Entropy is =  0.0
Reached leaf Node



In [9]:
Y_pred8 = clf.predict(x)
print("Predictions : ",Y_pred8)
print()

node data 0
val 0
node children {0: <__main__.TreeNode object at 0x000002B81D7BE630>, 1: <__main__.TreeNode object at 0x000002B81D7B3E80>}
node data 1
val 0
node children {0: <__main__.TreeNode object at 0x000002B81D7BE4A8>, 1: <__main__.TreeNode object at 0x000002B81D7BE208>}
node data 0
val 0
node children {0: <__main__.TreeNode object at 0x000002B81D7BE630>, 1: <__main__.TreeNode object at 0x000002B81D7B3E80>}
node data 1
val 1
node children {0: <__main__.TreeNode object at 0x000002B81D7BE4A8>, 1: <__main__.TreeNode object at 0x000002B81D7BE208>}
node data 0
val 1
node children {0: <__main__.TreeNode object at 0x000002B81D7BE630>, 1: <__main__.TreeNode object at 0x000002B81D7B3E80>}
node data 0
val 1
node children {0: <__main__.TreeNode object at 0x000002B81D7BE630>, 1: <__main__.TreeNode object at 0x000002B81D7B3E80>}
Predictions :  [0 1 1 1]



In [ ]:
from sklearn import datasets
X,Y = datasets.make_classification(n_samples=100, n_features=10, n_informative=2, n_redundant=2, random_state=0)

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
for i in range(len(X)):
    for j in range(len(X[0])):
        X[i][j] = int(X[i][j])
        
clf2 = DecisionTreeClassifier()
clf2.fit(X,Y)

In [ ]:
Y_pred2 = clf2.predict(X)
print("Predictions : ",Y_pred2)
print()
our_score = clf2.score(X,Y)
print("Score :",our_score) 
print()

In [ ]:
import sklearn.tree
clf3 = sklearn.tree.DecisionTreeClassifier()
clf3.fit(X,Y)
Y_pred3 = clf3.predict(X)
print(Y_pred3)
sklearn_score = clf3.score(X,Y)
print("Score :",sklearn_score)
print("Score of our model :",our_score)

print("Score of inbuilt sklearn's decision tree on the same data is:",sklearn_score)

In [10]:
de = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

In [12]:
da=de[0]

In [13]:
da[0]

0